In [ ]:
# 1. Uninstall ALL conflicting libraries
!pip uninstall -y numpy opencv-python opencv-python-headless ultralytics roboflow

# 2. Reinstall them in a strict order (Headless OpenCV is better for Kaggle)
!pip install "numpy<2.0" "opencv-python-headless<4.10" ultralytics roboflow

In [ ]:

from roboflow import Roboflow
rf = Roboflow(api_key="..........")
project = rf.workspace("..........").project("herlev-cervical-cells")
version = project.version(2)
dataset = version.download("folder")
                

loading Roboflow workspace...
loading Roboflow project...


In [2]:
import ultralytics
from ultralytics import YOLO

# Check if GPU is on
ultralytics.checks()

Ultralytics 8.3.236 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6567.8/8062.4 GB disk)


In [7]:
# Load a pre-trained classification model
model = YOLO('yolov8m-cls.pt')  # 'n' = nano (fastest), 's' = small (more accurate)

# Train it!
results = model.train(
    data='/kaggle/working/Herlev-Cervical-Cells-2', # Make sure this folder name matches exactly
    epochs=50, 
    imgsz=224, 
    batch=64,
    name='herlev_model',
    amp=False  # <--- ADD THIS LINE TO FIX THE ERROR
)

Ultralytics 8.3.236 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/Herlev-Cervical-Cells-2, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=herlev_model11, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspectiv

In [8]:
# Evaluate specifically on the Test set with the correct path
metrics = model.val(
    data='/kaggle/working/Herlev-Cervical-Cells-2',  # <--- Remind it where the data is!
    split='test'                                     # <--- Tell it to use the "test" folder
)

print(f"Final Test Accuracy: {metrics.top1:.3f}")

Ultralytics 8.3.236 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8m-cls summary (fused): 42 layers, 15,771,623 parameters, 0 gradients, 41.6 GFLOPs
train: /kaggle/working/Herlev-Cervical-Cells-2/train... found 1922 images in 7 classes ✅ 
val: /kaggle/working/Herlev-Cervical-Cells-2/valid... found 139 images in 7 classes ✅ 
test: /kaggle/working/Herlev-Cervical-Cells-2/test... found 137 images in 7 classes ✅ 
test: Fast image access ✅ (ping: 0.0±0.0 ms, read: 164.1±48.1 MB/s, size: 3.8 KB)
test: Scanning /kaggle/working/Herlev-Cervical-Cells-2/test... 137 images, 0 corrupt: 100% ━━━━━━━━━━━━ 137/137 356.7Kit/s 0.0s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 9/9 14.1it/s 0.6s0.2s
                   all      0.365      0.971
Speed: 0.5ms preprocess, 2.4ms inference, 0.1ms loss, 0.0ms postprocess per image
Results saved to /kaggle/working/runs/classify/val5
Final Test Accuracy: 0.365
